In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


Following the, Store Sales TS Forecasting - A Comprehensive Guide

In [3]:
#Basic imports
import numpy as np
import pandas as pd
import os
import gc
import warnings

In [4]:
#PACF - ACF
import statsmodels.api as sm

In [5]:
#Visualization tools
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [6]:
#config
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format #unclear, define this config
warnings.filterwarnings('ignore')

In [7]:
def readcsv(path):
    df = pd.read_csv(path)
    return df

In [8]:
train = readcsv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test = readcsv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
stores = readcsv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')

#transactions will be sorted with '.sort_values["~", "date"]'
transactions = readcsv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

### In time series, it is important to set dtype of time into 'datetime'. It allows time manitulation.

In [9]:
#Datetime 'pd.to_datetime(~.date)'
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
transactions["date"] = pd.to_datetime(transactions.date)

In [10]:
# 'astype' is a method of changing dataframe's dtype.
train.onpromotion = train.onpromotion.astype("float16")
train.sales = train.sales.astype("float32")
stores.cluster = stores.cluster.astype("int8")

In [11]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0.00
1,1,2013-01-01,1,BABY CARE,0.00,0.00
2,2,2013-01-01,1,BEAUTY,0.00,0.00
3,3,2013-01-01,1,BEVERAGES,0.00,0.00
4,4,2013-01-01,1,BOOKS,0.00,0.00


## Transactions

In [12]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


separate copy_df by store number

resample by date by W or D3

merge back into one dfresam_df = pd.DataFrame()

for i in range(1, 55):
    df = copy_group.get_group(i)
    df.set_index('date', inplace=True)
    df_resampled = df.resample('W').agg({'store_nbr': 'first', 'transactions': 'sum'})
    if i == 1:
        resam_df = df_resampled
    else:
        resam_df = pd.merge(resam_df, df_resampled, left_index=True, right_index=True, how='outer')

# Reset the index to make 'date' a regular column
resam_df.reset_index(inplace=True)


px.line new df.

In [17]:
copy_df = transactions.copy(deep=True)
copy_df['date'] = pd.to_datetime(copy_df['date'])
copy_group = copy_df.groupby('store_nbr')

resam_df = pd.DataFrame()

for i in range(1, 55):
    df = copy_group.get_group(i)
    df.set_index('date', inplace=True)
    df_resampled = df.resample('2D').agg({'store_nbr': 'first', 'transactions': 'sum'})
    resam_df = pd.concat([resam_df, df_resampled])

# Reset the index to make 'date' a regular column
resam_df.reset_index(inplace=True)



In [18]:
resam_df

,date,store_nbr,transactions
0,2013-01-02,1.00,3944
1,2013-01-04,1.00,3372
2,2013-01-06,1.00,2327
3,2013-01-08,1.00,3779
4,2013-01-10,1.00,3492
...,...,...,...
42223,2017-08-07,54.00,1594
42224,2017-08-09,54.00,1418
42225,2017-08-11,54.00,1671
42226,2017-08-13,54.00,1872


In [19]:
temp = pd.merge(train.groupby(["date", "store_nbr"]).sales.sum().reset_index(), transactions, how = "left")
print("Spearman Correlation between Total Sales and Transactions: {:,.4f}".format(temp.corr("spearman").sales.loc["transactions"]))


px.line(resam_df.sort_values(["store_nbr", "date"]), x='date', y='transactions', color='store_nbr',title = "Transactions" )

Spearman Correlation between Total Sales and Transactions: 0.8175


In [20]:
b = transactions.copy(deep=True)
b['year'] = b.date.dt.year
b['month'] = b.date.dt.month
px.box(b, x='year', y='transactions', color='month', title="Transactions")

In [22]:
a = transactions.set_index('date').resample('M').transactions.mean().reset_index()
a['year'] = a.date.dt.year
px.line(a, x='date', y='transactions', color='year', title = 'Monthly Avg Trans')